In [16]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.model_selection import learning_curve

Reading the data and checking file data type is missing values 

In [17]:
train_filepath = "C:/Users/user/VSCode/DATA/OLC/train.csv"
train_data = pd.read_csv(train_filepath)
train_data.info()
data_map = train_data.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5346 entries, 0 to 5345
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           5346 non-null   int64  
 1   Churn                        5346 non-null   int64  
 2   Tenure                       5094 non-null   float64
 3   PreferredLoginDevice         5346 non-null   object 
 4   CityTier                     5346 non-null   int64  
 5   WarehouseToHome              5109 non-null   float64
 6   PreferredPaymentMode         5346 non-null   object 
 7   Gender                       5346 non-null   object 
 8   HourSpendOnApp               5107 non-null   float64
 9   NumberOfDeviceRegistered     5346 non-null   int64  
 10  PreferedOrderCat             5346 non-null   object 
 11  SatisfactionScore            5346 non-null   int64  
 12  MaritalStatus                5346 non-null   object 
 13  NumberOfAddress   

Checking the data descriptive statistics

In [ ]:
train_data.describe()

Label Encoding the categorical value

In [18]:
data_map["PreferredLoginDevice"] = data_map["PreferredLoginDevice"].map({
    "Phone" : 0,
    "Mobile Phone" : 0,
    "Computer" : 1
})
data_map["PreferredPaymentMode"] = data_map["PreferredPaymentMode"].map({
    "Cash on Delivery" : 0,
    "COD" : 0,
    "CC" : 1,
    "Credit Card" : 1,
    "Debit Card" : 2,
    "E wallet" : 3,
    "UPI" : 4
})
data_map["Gender"] = data_map["Gender"].map({
    "Male" : 0,
    "Female" : 1
}) 
data_map["PreferedOrderCat"] = data_map["PreferedOrderCat"].map({
    "Fashion" : 0,
    "Grocery" : 1,
    "Laptop & Accessory" : 2,
    "Mobile" : 3,
    "Mobile Phone" : 3,
    "Others" : 4
})
data_map["MaritalStatus"] = data_map["MaritalStatus"].map({
    "Single" : 0,
    "Married" : 1,
    "Divorced" : 2
})

Correlation Matrix to find correlation of each variables

In [ ]:
corr_matrix = data_map.corr()
plt.figure(figsize=(17,15))
sns.heatmap(corr_matrix, annot=True, square=True, cmap='coolwarm')
plt.title('Correlation Matrix of Every Variables', fontsize=20)
plt.show()

Boxplot to find the outliers and the data spread

In [ ]:
num_columns = data_map.select_dtypes(include=['int64', 'float64']).columns
num_columns = num_columns.drop(['id'])
# Set up subplots with more space between subplots
num_plots = len(num_columns)
fig, axes = plt.subplots(nrows=1, ncols=num_plots, figsize=(45, 10))

# Create box plots for each numerical column in a subplot
for i, column in enumerate(num_columns):
    sns.boxplot(y=column, data=data_map, ax=axes[i])
    axes[i].set_title(f'Box plot for {column}')

plt.tight_layout(pad=5.0)  # Adjust the pad parameter for spacing between subplots
plt.show()


Pairplot of every single combination of variables

In [ ]:
sns.pairplot(data_map)
plt.show()

Density Plot to chek the variable's density compared to churn variable

In [ ]:
num_plots = len(num_columns)
num_cols = 2  # Number of columns in each row
num_rows = num_plots // num_cols + (num_plots % num_cols > 0)  # Calculate the number of rows needed
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(35, 15))

axes = axes.flatten()

for i, column in enumerate(num_columns):
    sns.kdeplot(data=data_map, x=column, hue="Churn", fill=True, common_norm=False, ax=axes[i])
    axes[i].set_title(f'Density plot of {column} grouped by Churn')

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

Using MICE (Multiple Imputation by Chained Equations) to impute missing values

In [19]:
data_map = data_map.drop('id', axis=1)
# Create the imputer
imputer = IterativeImputer(random_state=42)
# Fit the imputer and transform the data
data_imputed = pd.DataFrame(imputer.fit_transform(data_map), columns=data_map.columns)

Inputting the test data

In [20]:
test_filepath = "C:/Users/user/VSCode/DATA/OLC/test.csv"
test = pd.read_csv(test_filepath)
test_data = test.copy()

Label Encoding the test data

In [21]:
test_data["PreferredLoginDevice"] = test_data["PreferredLoginDevice"].map({
    "Phone" : 0,
    "Mobile Phone" : 0,
    "Computer" : 1
})
test_data["PreferredPaymentMode"] = test_data["PreferredPaymentMode"].map({
    "Cash on Delivery" : 0,
    "COD" : 0,
    "CC" : 1,
    "Credit Card" : 1,
    "Debit Card" : 2,
    "E wallet" : 3,
    "UPI" : 4
})
test_data["Gender"] = test_data["Gender"].map({
    "Male" : 0,
    "Female" : 1
}) 
test_data["PreferedOrderCat"] = test_data["PreferedOrderCat"].map({
    "Fashion" : 0,
    "Grocery" : 1,
    "Laptop & Accessory" : 2,
    "Mobile" : 3,
    "Mobile Phone" : 3,
    "Others" : 4
})
test_data["MaritalStatus"] = test_data["MaritalStatus"].map({
    "Single" : 0,
    "Married" : 1,
    "Divorced" : 2
})

test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           284 non-null    int64  
 1   Tenure                       272 non-null    float64
 2   PreferredLoginDevice         284 non-null    int64  
 3   CityTier                     284 non-null    int64  
 4   WarehouseToHome              270 non-null    float64
 5   PreferredPaymentMode         284 non-null    int64  
 6   Gender                       284 non-null    int64  
 7   HourSpendOnApp               268 non-null    float64
 8   NumberOfDeviceRegistered     284 non-null    int64  
 9   PreferedOrderCat             284 non-null    int64  
 10  SatisfactionScore            284 non-null    int64  
 11  MaritalStatus                284 non-null    int64  
 12  NumberOfAddress              284 non-null    int64  
 13  Complain            

Imputing the test data using the same method as for the train data

In [22]:
test_data = test_data.drop('id', axis=1)
# Create the imputer
imputer = IterativeImputer(random_state=42)

# Fit the imputer and transform the data
test_imputed = pd.DataFrame(imputer.fit_transform(test_data), columns=test_data.columns)

Dropping churn from training (X) and put it as target (y)

Split the train data into training and testing

In [23]:
X = data_imputed.drop('Churn', axis=1)
y = data_imputed['Churn']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Using Stacking to combine both model

In [ ]:
# # Define the base models
# base_models = [
#     ('rf', RandomForestClassifier(
#         n_estimators=500,
#         max_depth=60,
#         min_samples_split=2,
#         min_samples_leaf=1,
#         bootstrap=False
#     )),
#     ('xgb', xgb.XGBClassifier(
#         n_estimators=700, 
#         learning_rate=0.2,
#         max_depth=8,
#         min_child_weight=1,
#         gamma=0,
#         subsample=0.8,
#         colsample_bytree=1.0,
#         random_state=42,
#     ))
# ]

# # Split the data
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # # Define hyperparameter grids for each base model
# # param_grid_rf = {
# #     'n_estimators': [400, 500, 600, 700, 800],
# #     'max_depth': [60, 65, 70, 80],
# #     'min_samples_split': [2, 5, 10],
# #     'min_samples_leaf': [1, 2, 4],
# #     'bootstrap': [False, True]
# # }

# # param_grid_xgb = {
# #     'n_estimators': [600, 700, 800, 900],
# #     'learning_rate': [0.1, 0.2, 0.3, 0.5],
# #     'max_depth': [6, 8, 10, 20, 30],
# #     'min_child_weight': [1, 2, 4],
# #     'gamma': [0, 0.1, 0.3, 0.5],
# #     'subsample': [0.8, 0.9, 1.0],
# #     'colsample_bytree': [1.0],
# # }

# # # Iterate over base models and perform hyperparameter tuning
# # for (name, model), param_grid in zip(base_models, [param_grid_rf, param_grid_xgb]):
# #     grid_search_base = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='f1', verbose=2, n_jobs=-1)
# #     grid_search_base.fit(X_train, y_train)
    
# #     # Print the best parameters and score for each base model
# #     print(f'Best Parameters for {name}: {grid_search_base.best_params_}')
# #     print(f'Best Score for {name}: {grid_search_base.best_score_}')

# #     # Update the base model with the best parameters
# #     model.set_params(**grid_search_base.best_params_)
    
# #     model.fit(X_train, y_train)

# # Iterate over base models and perform hyperparameter tuning
# for (name, model) in base_models:
#     model.fit(X_train, y_train)

# # Create the stacking model with SVM as the final estimator
# meta_model_svm = SVC(random_state=42)
# stacking_model_svm = StackingClassifier(estimators=base_models, final_estimator=meta_model_svm)

# # Define hyperparameter grid for stacking model with SVM
# param_dist_svm = {
#     'final_estimator__C': uniform(60, 120),  # Narrow down the range for C
#     'final_estimator__kernel': ['linear', 'poly'],
#     'final_estimator__gamma': ['scale', 'auto', 0.001, 0.01, 0.1],  # Fine-tune gamma
#     'final_estimator__class_weight': [None, 'balanced'],
#     'final_estimator__degree': [3, 4, 5, 6, 7, 8],  # Fine-tune the degree parameter
#     'final_estimator__coef0': [0.1, 0.5, 1.0],  # Fine-tune the coef0 parameter
#     'final_estimator__probability': [True, False]
# }

# # Create the RandomizedSearchCV object for stacking model with SVM
# random_search_stacking_svm = RandomizedSearchCV(
#     estimator=stacking_model_svm,
#     param_distributions=param_dist_svm,
#     n_iter= 50,  # Adjust the number of iterations
#     cv=5,
#     scoring='f1',
#     verbose=2,
#     n_jobs=-1,
#     random_state=42
# )

# # Fit the RandomizedSearchCV object to the data
# random_search_stacking_svm.fit(X_train, y_train)

# # Print the best parameters and best score for stacking model with SVM
# print(f'Best Parameters for Stacking Model with SVM: {random_search_stacking_svm.best_params_}')
# print(f'Best Score for Stacking Model with SVM: {random_search_stacking_svm.best_score_}')

# # Make predictions and evaluate the stacking model with SVM on the validation set
# stacking_predictions_svm = random_search_stacking_svm.best_estimator_.predict(X_val)

# accuracy_stacking_svm = accuracy_score(y_val, stacking_predictions_svm)
# precision_stacking_svm = precision_score(y_val, stacking_predictions_svm)
# recall_stacking_svm = recall_score(y_val, stacking_predictions_svm)
# confusion_matrix_stacking_svm = confusion_matrix(y_val, stacking_predictions_svm)

# print(f"Accuracy of Stacking Model with SVM on Validation Set: {accuracy_stacking_svm}")
# print(f"Precision of Stacking Model with SVM on Validation Set: {precision_stacking_svm}")
# print(f"Recall of Stacking Model with SVM on Validation Set: {recall_stacking_svm}")
# print(f"Confusion Matrix of Stacking Model with SVM on Validation Set:\n{confusion_matrix_stacking_svm}")

In [24]:
# Define the base models
base_models = [
    ('rf', RandomForestClassifier(
        n_estimators=500,
        max_depth=70,
        min_samples_split=2,
        min_samples_leaf=1,
        bootstrap=False
    )),
    ('xgb', xgb.XGBClassifier(
        n_estimators=700, 
        learning_rate=0.2,
        max_depth=8,
        min_child_weight=1,
        gamma=0,
        subsample=1,
        colsample_bytree=1.0,
        random_state=42,
    ))
]

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Iterate over base models and perform hyperparameter tuning
for (name, model) in base_models:
    model.fit(X_train, y_train)

# Create the stacking model with SVM as the final estimator
meta_model_svm = SVC(C= 114.05991023634516, 
                     class_weight = 'balanced', 
                     coef0 = 0.1, 
                     degree = 6, 
                     gamma = 'auto', 
                     kernel = 'poly',
                     probability= True)

stacking_model_svm = StackingClassifier(estimators=base_models, final_estimator=meta_model_svm)

# Fit the stacking model with SVM to the data
stacking_model_svm.fit(X_train, y_train)

# Make predictions and evaluate the stacking model with SVM on the validation set
stacking_predictions_svm = stacking_model_svm.predict(X_val)

accuracy_stacking_svm = accuracy_score(y_val, stacking_predictions_svm)
precision_stacking_svm = precision_score(y_val, stacking_predictions_svm)
recall_stacking_svm = recall_score(y_val, stacking_predictions_svm)
confusion_matrix_stacking_svm = confusion_matrix(y_val, stacking_predictions_svm)

print(f"Accuracy of Stacking Model with SVM on Validation Set: {accuracy_stacking_svm}")
print(f"Precision of Stacking Model with SVM on Validation Set: {precision_stacking_svm}")
print(f"Recall of Stacking Model with SVM on Validation Set: {recall_stacking_svm}")
print(f"Confusion Matrix of Stacking Model with SVM on Validation Set:\n{confusion_matrix_stacking_svm}")


Accuracy of Stacking Model with SVM on Validation Set: 0.9813084112149533
Precision of Stacking Model with SVM on Validation Set: 0.9390243902439024
Recall of Stacking Model with SVM on Validation Set: 0.9390243902439024
Confusion Matrix of Stacking Model with SVM on Validation Set:
[[896  10]
 [ 10 154]]


In [27]:
# Generate Predictions for Validation Set
base_model_predictions_val = {name: model.predict(X_val) for name, model in base_models}
base_model_predictions_array_val = np.column_stack(list(base_model_predictions_val.values()))

# Fit the stacking model with SVM
stacking_model_svm.fit(base_model_predictions_array_val, y_val)

# Make Final Predictions
stacking_predictions = stacking_model_svm.predict(base_model_predictions_array_val)

# Evaluate Stacking Performance
accuracy_stacking = accuracy_score(y_val, stacking_predictions)
print(f"Accuracy of Stacking: {accuracy_stacking}")


Accuracy of Stacking: 0.9841121495327103


Checking the cross val of the stacking model

In [28]:
#Apply cross-validation on the predicted test labels
cross_val_scores = cross_val_score(stacking_model_svm, base_model_predictions_array_val, y_val, cv=5, scoring='accuracy')

#Print cross-validated scores
print("Cross-validated Accuracy Scores:", cross_val_scores)
print("Mean Accuracy:", np.mean(cross_val_scores))

Cross-validated Accuracy Scores: [0.99065421 0.98130841 0.98598131 0.97663551 0.97663551]
Mean Accuracy: 0.9822429906542057


Use the final model to predict test data

In [29]:
base_model_test_predictions = {name: model.predict(test_imputed) for name, model in base_models}

# Convert dictionary values to a NumPy array
base_model_test_predictions_array = np.column_stack(list(base_model_test_predictions.values()))

# Make final predictions on the test data using the stacking model
stacking_test_predictions = stacking_model_svm.predict(base_model_test_predictions_array)
display(stacking_test_predictions)

array([1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
       0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
       1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 1., 0.

In [30]:
val = pd.read_csv("C:/Users/user/VSCode/PYTHON/results(hmm).csv")
val = val.drop('id', axis=1)

In [31]:
from sklearn.metrics import confusion_matrix

# Print confusion matrix
conf_matrix = confusion_matrix(val, stacking_test_predictions)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[165   0]
 [  4 115]]


Save the result with id to csv file



In [32]:
test_imputed['Prediction'] = stacking_test_predictions
test_imputed['id'] = test['id']
# Save the DataFrame to a CSV file, including only the 'ID' and 'Prediction' columns
test_imputed[['id', 'Prediction']].to_csv('results(huh).csv', index=False)

Creating Learning Curve Graph to analyze model performance changes from data size

In [ ]:
# Initialize Stacking Classifier with the Meta Learner
clf = random_search_stacking_svm.best_estimator_

# Generate a learning curve
train_sizes, train_scores, test_scores = learning_curve(clf, X, y, cv=5)

# Calculate mean and standard deviation for training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Calculate mean and standard deviation for test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve
plt.plot(train_sizes, train_mean, label='Training score')
plt.plot(train_sizes, test_mean, label='Cross-validation score')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='#DDDDDD')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color='#DDDDDD')

# Create plot
plt.title('Learning Curve')
plt.xlabel('Training Size')
plt.ylabel('Accuracy Score')
plt.legend(loc='best')
plt.grid()
plt.show()